In [ ]:
import os
import pandas as pd
import numpy as np
import preprocessor as prep
import postprocessor as postp
import processor as proc
import pandas as pd
import unifloc.uniflocpy.uTools.plotly_workflow as pw
import datetime
from sklearn import metrics
import multiprocessing as mp
from multiprocessing import Pool
from preprocessor import load_and_edit_chess_data
import seaborn as sns

from sklearn import metrics
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.decomposition import PCA
from pandas.api.types import is_numeric_dtype

Здесь я хочу избавиться от "временной метки", она сильно мешает автоматической обработке данных

Вместо неё в ячейке ниже есть `mark` - то что припишется в конце папки

In [ ]:
mark = ''
left_boundary = [datetime.datetime(2019,1,1)]#, datetime.datetime(2018,11,29)]
right_boundary = [datetime.datetime(2019,2,28)]#, datetime.datetime(2019,2,28)]

In [ ]:
time_to_resamle = '3h'
created_input_data_type = 0 
well_names = ['569', '601', '1354', '1567', '1602', '1628']
for well_name in well_names:
    path_to_well = f'data/{well_name}'
    cs_data_filename = f'{path_to_well}/init_edit/{well_name}_first_edit.csv'
    edited_data_cs = prep.load_and_edit_cs_data(cs_data_filename=cs_data_filename,
                                                time_to_resamle=time_to_resamle, 
                                                created_input_data_type=created_input_data_type
                                               )
    chess_data = prep.load_and_edit_chess_data(f'{path_to_well}/Скв. {well_name} (01.08.2018-28.02.2019).xls',
                                               time_to_resamle)
    
    path_to_input_data = f'{path_to_well}/adapt_input'+mark
    try:
        os.mkdir(path_to_input_data)
    except:
        pass
    created_input_data = edited_data_cs.join(chess_data, how = 'outer')
    created_input_data = created_input_data.dropna(subset = ['Объемный дебит жидкости (СУ)'])
    created_input_data = prep.cut_df(created_input_data, left_boundary, right_boundary)
    plot_file_path = f'{path_to_input_data}/{well_name}_adapt_input.html'
    input_data_traces = pw.create_traces_list_for_all_columms(created_input_data, 'lines+markers', use_gl = True)
    pw.plot_subplots(input_data_traces, plot_file_path, True)

    created_input_data.to_csv(f'data/{well_name}/adapt_input/{well_name}_adapt_input.csv')
    created_input_data['Линейное давление (СУ)'] =  created_input_data['Линейное давление (СУ)'] * 10
    
    qliq = {'Объемный дебит жидкости (СУ)':['Объемный дебит жидкости (СУ)']}
    pressure_intake = {'Давление на приеме': ['Давление на приеме насоса (пласт. жидкость) (СУ)']}
    pressure_wh = {'Рлин ТМ (Ш)': ['Рлин ТМ (Ш)', 'Линейное давление (СУ)']}
    pressure_bf = {'Рбуф (Ш)': ['Рбуф (Ш)']}
    temp_intake = {'Температура на приеме насоса (пласт. жидкость) (СУ)': ['Температура на приеме насоса (пласт. жидкость) (СУ)']}
    frequencies = {'Частота, Гц':
                   ['F вращ ТМ (Ш)', 'Выходная частота ПЧ (СУ)']}
    choke = {'Размер штуцера, мм': ['Dшт (Ш)']}
    power = {'Активная мощность (СУ)':['Активная мощность (СУ)']}
    voltage = {'Напряжение на выходе ТМПН (СУ)':['Напряжение на выходе ТМПН (СУ)']}
    cos = {'Коэффициент мощности (СУ)':['Коэффициент мощности (СУ)']}
    gor = {'ГФ (СУ)':['ГФ (СУ)']}
    wc = {'Процент обводненности (СУ)':['Процент обводненности (СУ)']}
    all_banches = [qliq, gor,wc, pressure_intake, pressure_wh, pressure_bf,
                   temp_intake, frequencies, choke,power,voltage,cos]
    
    plot_file_path = f'{path_to_input_data}/{well_name}_adapt_input_report.html.html'
    pw.create_report_html(created_input_data, all_banches, plot_file_path)

Фиксим ресэмпл

In [ ]:
prep.read_tr_and_get_data(os.getcwd() + '\\data\\tr\\' + "Техрежим, , февраль 2019.xls", '569').esp_name_str